Продолжим эксперименты с таблицами и связями между ними. В нашей фильмотеке каждое произведение может относиться к одному из типов: мультфильм, фильм, сериал или мультсериал. 

![alt text](https://pictures.s3.yandex.net/resources/S2_156_1682533755.png)

Перечень типов сохраним в таблице **product_types**; к этой таблице должна «тянуться» связь от **video_products**. 

Не будем перегружать таблицу **video_products** лишними полями: забудем на время про поле `original_title_id` и сосредоточимся на связи с **product_types**.

В таблице video_products создадим FK-поле `type_id`.

![alt text](https://pictures.s3.yandex.net/resources/S02_226_1685551234.png)


Кинолента может относиться только к одному типу; для любого фильма обязательно должен быть указан его тип — значение поля `type_id` не может быть `NULL`. 

К одному типу может относиться много фильмов из таблицы **video_products**. Но может оказаться и так, что тип фильма есть, но в таблице **video_products** нет ни одного фильма такого типа.

Это связь «многие к одному»: «много фильмов привязаны к одному типу».

В SQL создание такой связи описывается так: 

```sql
CREATE TABLE IF NOT EXISTS video_products(
...
type_id INTEGER NOT NULL,
FOREIGN KEY(type_id) REFERENCES product_types(id)
...);
```

В переводе на русский:

* создай поле `type_id` целочисленного типа; это поле не должно быть пустым: `NOT NULL`;

* объяви это поле внешним ключом, связанным с первичным ключом другой таблицы — с полем `id` таблицы **product_types**.

Для поля `type_id` не указан параметр `UNIQUE`; таким образом, несколько записей из **video_products** могут ссылаться на одну и ту же запись в **product_types**. 

В Python создание связанных таблиц **video_products** и **product_types** будет выглядеть так:

In [1]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

cur.executescript('''
CREATE TABLE IF NOT EXISTS product_types(
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);
CREATE TABLE IF NOT EXISTS video_products(
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    type_id INTEGER NOT NULL,
    FOREIGN KEY(type_id) REFERENCES product_types(id)
);
''')

con.close()

Заполним таблицы данными; значение поля `type_id` в каждой записи таблицы video_products — это `id` определённой записи в таблице product_types:

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

video_products = [
    (1, 'Безумные мелодии Луни Тюнз', 2),
    (2, 'Весёлые мелодии', 2),
    (3, 'Кто подставил кролика Роджера', 3),
    (4, 'Хороший, плохой, злой', 3),
    (5, 'Последний киногерой', 3),
    (6, 'Она написала убийство', 4),
    (7, 'Миссис Харрис едет в Париж', 3),
]
product_types = [
    (1, 'Мультфильм'),
    (2, 'Мультсериал'),
    (3, 'Фильм'),
    (4, 'Сериал'),
]
cur.executemany('INSERT INTO product_types VALUES(?, ?);', product_types)
cur.executemany('INSERT INTO video_products VALUES(?, ?, ?);', video_products)

con.commit()
con.close()

Выбрать все названия кинолент из категории 'Фильм' можно таким запросом: 

```sql
SELECT video_products.name,
       product_types.name
FROM video_products,
     product_types
WHERE video_products.type_id = product_types.id AND product_types.name = 'Фильм';
```

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

cur.executescript('''
CREATE TABLE IF NOT EXISTS categories (
    id INTEGER PRIMARY KEY,
    slug TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS ice_cream(
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    description TEXT,
    price REAL NOT NULL,
    category_id INTEGER NOT NULL,
    FOREIGN KEY(category_id) REFERENCES categories(id)
);
''')

con.close()

In [ ]:
import sqlite3

con = sqlite3.connect('db.sqlite')
cur = con.cursor()

results = cur.execute('''
SELECT ice_cream.title,
       categories.slug,
       MAX(ice_cream.price)
FROM ice_cream,
     categories
WHERE 
    ice_cream.category_id = categories.id
GROUP BY categories.slug
ORDER BY ice_cream.price DESC;
''')

for result in results:
    print(result)

con.close()